In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score

# Entrenar

In [5]:
data = pd.read_csv('../age_gender.csv')

In [6]:
data['pixels'] = data['pixels'].apply(lambda x: np.array(x.split(), dtype='float32').reshape(48, 48))

In [7]:
X = np.stack(data['pixels'].values) / 255.0
X = X.reshape(-1, 48, 48, 1)

In [8]:
y = data['gender'].values

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [38]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(y.shape[1], activation='softmax')
])

C:\My programers\Anaconda3\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [50]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [72]:
history = model.fit(X_train, y_train, epochs=15, batch_size=32, validation_split=0.2)

Epoch 1/15
475/475 ━━━━━━━━━━━━━━━━━━━━ 19s 37ms/step - loss: 484.7908 - mae: 16.8807 - val_loss: 251.0229 - val_mae: 12.3882
Epoch 2/15
475/475 ━━━━━━━━━━━━━━━━━━━━ 17s 36ms/step - loss: 272.9424 - mae: 12.6517 - val_loss: 235.3226 - val_mae: 11.3162
Epoch 3/15
475/475 ━━━━━━━━━━━━━━━━━━━━ 17s 37ms/step - loss: 246.9290 - mae: 11.9175 - val_loss: 223.3092 - val_mae: 10.9336
Epoch 4/15
475/475 ━━━━━━━━━━━━━━━━━━━━ 18s 37ms/step - loss: 235.7262 - mae: 11.6183 - val_loss: 187.9998 - val_mae: 10.1302
Epoch 5/15
475/475 ━━━━━━━━━━━━━━━━━━━━ 18s 37ms/step - loss: 210.6188 - mae: 10.9956 - val_loss: 187.4032 - val_mae: 9.9988
Epoch 6/15
475/475 ━━━━━━━━━━━━━━━━━━━━ 18s 37ms/step - loss: 200.6346 - mae: 10.6949 - val_loss: 186.8240 - val_mae: 10.0053
Epoch 7/15
475/475 ━━━━━━━━━━━━━━━━━━━━ 18s 37ms/step - loss: 187.6526 - mae: 10.3241 - val_loss: 161.6332 - val_mae: 9.2268
Epoch 8/15
475/475 ━━━━━━━━━━━━━━━━━━━━ 18s 37ms/step - loss: 176.6841 - mae: 9.9597 - val_loss: 145.3296 - val_mae: 8.7

In [73]:
model.save('gender_model.h5')

# Evaluar

In [24]:
model = load_model("../modelos/gender_model.h5")

In [14]:
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(y, axis=1)
accuracy = accuracy_score(true_classes, predicted_classes)
correct_predictions = (predicted_classes == true_classes).sum()

In [26]:
print(f'Accuracy: {accuracy * 100:.2f}%')
print(f'Correct Predictions: {correct_predictions}/{len(y)}')

Accuracy: 90.04%
Correct Predictions: 21344/23705


In [15]:
def predecir(imagen):
    pixels = imagen['pixels'].values[0]
    if isinstance(pixels, str):
        pixels = np.array([int(i) for i in pixels.split()])
    elif isinstance(pixels, np.ndarray):
        pixels = pixels.astype(int)
    pixels = pixels.reshape(48, 48)
    pixels = np.expand_dims(pixels, axis=(0, -1)) / 255.0
    return model.predict(pixels)[0][0]

In [16]:
gender_dict = {0: 'Male', 1: 'Female'}

In [27]:
random_row = data.sample(1)
prediction = predecir(random_row)
predicted_class = int(prediction > 0.5)

predicted_gender = gender_dict[predicted_class]
real_gender = random_row['gender'].values[0]
print(f'Real Gender: {gender_dict[real_gender]}, Predicted Gender: {predicted_gender}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step
Real Gender: Female, Predicted Gender: Female


In [28]:
random_row = data.sample(1)
prediction = predecir(random_row)
predicted_class = int(prediction > 0.5)

predicted_gender = gender_dict[predicted_class]
real_gender = random_row['gender'].values[0]
print(f'Real Gender: {gender_dict[real_gender]}, Predicted Gender: {predicted_gender}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Real Gender: Male, Predicted Gender: Male
